## Exploring and analysing our diverse porfolio data

### Introduction
In this notebook we will import our data, we are using from 3 indices alongside the Bank of England base rate.

Our Indices include: iShares MCSI World Index (GBP Hedged Acc), S&P 500 (GBP Hedged Acc) and Vanguard's Global Bond Index (GBP Hedged Acc)

We will simulate investing into equities, bonds and cash using these options.

### Importing Libraries and our data

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
#Parsing Data for csv files
iShares_World = pd.read_csv("Data//iShares MCSI World Index Data.csv", index_col=0, parse_dates=True, dayfirst=True)
iShares_SNP500 = pd.read_csv("Data//iShares SNP500 Index Data.csv", index_col=0, parse_dates=True, dayfirst=True)
Vanguard_Global_Bond = pd.read_csv("Data//Vanguard Global Bond Data.csv",index_col=0, parse_dates=True, dayfirst=True)
BoE_Rate = pd.read_csv("Data//BoE Interest Rate Data.csv", index_col=0, parse_dates=True, dayfirst=True)

C:\Users\allan\AppData\Local\Temp\ipykernel_10764\807073608.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  iShares_World = pd.read_csv("Data//iShares MCSI World Index Data.csv", index_col=0, parse_dates=True, dayfirst=True)
C:\Users\allan\AppData\Local\Temp\ipykernel_10764\807073608.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  iShares_SNP500 = pd.read_csv("Data//iShares SNP500 Index Data.csv", index_col=0, parse_dates=True, dayfirst=True)
C:\Users\allan\AppData\Local\Temp\ipykernel_10764\807073608.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Vanguard_Glob

### Combining our data into a single DataFrame
Before simulating our investment we need to collate all the data into a single dataset

In [3]:
#Renaming columns and reducing down the dataframes to only the required columns
iShares_World = iShares_World.rename(columns={"Close": "iShares_World_Close"})
iShares_SNP500 = iShares_SNP500.rename(columns={"Close": "iShares_SNP500_Close"})
Vanguard_Global_Bond = Vanguard_Global_Bond.rename(columns={"Close": "Vanguard_Global_Bond_Close"})
BoE_Rate = BoE_Rate.rename(columns={"Rate": "BoE_Rate"})

iShares_World = iShares_World["iShares_World_Close"]
iShares_SNP500 = iShares_SNP500["iShares_SNP500_Close"]
Vanguard_Global_Bond = Vanguard_Global_Bond[ "Vanguard_Global_Bond_Close"]


In [4]:
#Merging the dataframes into one and removing commas from the numbers
merged = pd.merge(iShares_World,iShares_SNP500,on="Date")
merged = pd.merge(merged,Vanguard_Global_Bond,on="Date")
merged = pd.merge(merged,BoE_Rate,on="Date")

merged = merged.replace(',','', regex=True)

In [5]:
#Checking the data types
merged.dtypes

iShares_World_Close            object
iShares_SNP500_Close           object
Vanguard_Global_Bond_Close    float64
BoE_Rate                      float64
dtype: object

In [6]:
#Changing objects to float variables and checking
merged[["iShares_World_Close", "iShares_SNP500_Close"]]=merged[["iShares_World_Close", "iShares_SNP500_Close"]].astype(float)
merged.dtypes

iShares_World_Close           float64
iShares_SNP500_Close          float64
Vanguard_Global_Bond_Close    float64
BoE_Rate                      float64
dtype: object

### Creating our functions
Below we have our 3 functions, the first simulates a monthly investment of £100 over 10 years, the second £1200 per year and the final is again a monthly investment but rebalancing the portfolio at the start of each year

In [7]:
#Setting our desired date range and creating a list of the 1st of Jan of each year to use in our functions
date_range = pd.date_range('2015-01-01', '2024-12-31', freq='MS')
year_starts = pd.date_range('2015-01-01', '2024-12-31', freq='YS')

In [8]:
#Next we create a fuction to simulate our investment, our function takes inputs of the proportion of the investment going into each asset, 
#then invests £100 per month at these proportions for 10 years from the start of 2005 to then end of 2024

def invest_monthly(iShares_World_prop,iShares_SNP500_prop,Vanguard_Global_Bond_prop,Cash_prop):

    #Creating our variables and zeroing them off
    total_shares = 0
    total_invested = 0
    iShares_World_total_shares = 0
    iShares_SNP500_total_shares = 0
    Vanguard_Global_Bond_total_shares = 0
    Cash_total = 0
    df = pd.DataFrame({'Date': [], 'iShares_World_Value': [], 'iShares_SNP500_Value': [],
                       "Vanguard_Global_Bond_Value":[],"Cash_total":[], "Total_Value":[]}).set_index("Date")
    
    #Next we cycle throuch all the dates, extracing the close price on each date (month starts) and calculating the new amount of shares + value
    for date in date_range:
        #iShares World
        iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
        iShares_World_total_shares =  iShares_World_total_shares + (100*iShares_World_prop/iShares_World_current_close)
        iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
        
        #iShares SNP500
        iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
        iShares_SNP500_total_shares =  iShares_SNP500_total_shares + (100*iShares_SNP500_prop/iShares_SNP500_current_close)
        iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
        
        #Vanguard
        Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
        Vanguard_Global_Bond_total_shares =  Vanguard_Global_Bond_total_shares + (100*Vanguard_Global_Bond_prop/Vanguard_Global_Bond_current_close)
        Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
        
        #For cash we apply the bank of England base rate (divided by 12 being monthly)
        BoE_current_rate = merged.loc[date,"BoE_Rate"]
        BoE_current_rate_monthly = BoE_current_rate/12
        Cash_total = (100*Cash_prop) + (BoE_current_rate_monthly*Cash_total/100) + Cash_total
        Total_Value=(iShares_World_total_shares*iShares_World_current_close + 
                     iShares_SNP500_total_shares*iShares_SNP500_current_close + 
                     Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close + Cash_total)
    
        #Creating a new row and adding it to the bottom of our new datafram
        newrow = pd.DataFrame({'Date': [date], 'iShares_World_Value': [iShares_World_Value], 'iShares_SNP500_Value': [iShares_SNP500_Value],
        "Vanguard_Global_Bond_Value":[Vanguard_Global_Bond_Value],"Cash_total":[Cash_total], "Total_Value": [Total_Value]}).set_index("Date")
        df = pd.concat([df, newrow])
   
    #Returning a dataframe with all the values
    results = df
    return results



In [10]:
#Similar again to monthly invest, we use an if statement to check if its a year start, then invest £1200, otherwise nothing is invested
def invest_yearly(iShares_World_prop,iShares_SNP500_prop,Vanguard_Global_Bond_prop,Cash_prop):

    total_shares = 0
    total_invested = 0
    
    iShares_World_total_shares = 0
    iShares_SNP500_total_shares = 0
    Vanguard_Global_Bond_total_shares = 0
    Cash_total = 0
    Total_Value = 0
    
    df = pd.DataFrame({'Date': [], 'iShares_World_Value': [], 'iShares_SNP500_Value': [],
                       "Vanguard_Global_Bond_Value":[],"Cash_total":[], "Total_Value":[]}).set_index("Date")
    
    
    for date in date_range:
        if date in year_starts:
            #iShares World
            iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
            iShares_World_total_shares =  iShares_World_total_shares + (1200*iShares_World_prop/iShares_World_current_close)
            iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
            
            #iShares SNP500
            iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
            iShares_SNP500_total_shares =  iShares_SNP500_total_shares + (1200*iShares_SNP500_prop/iShares_SNP500_current_close)
            iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
            
            #Vanguard Global Bond
            Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
            Vanguard_Global_Bond_total_shares =  Vanguard_Global_Bond_total_shares + (1200*Vanguard_Global_Bond_prop/Vanguard_Global_Bond_current_close)
            Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
            
            #Cash
            BoE_current_rate = merged.loc[date,"BoE_Rate"]
            BoE_current_rate_monthly = BoE_current_rate/12
            Cash_total = (1200*Cash_prop) + (BoE_current_rate_monthly*Cash_total/100) + Cash_total
        else:
            #iShares World
            iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
            iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
            
            #iShares SNP500
            iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
            iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
            
            #Vanguard Global Bond
            Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
            Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
            
            #Cash
            BoE_current_rate = merged.loc[date,"BoE_Rate"]
            BoE_current_rate_monthly = BoE_current_rate/12
            Cash_total = (BoE_current_rate_monthly*Cash_total/100) + Cash_total            
        
        Total_Value=(iShares_World_total_shares*iShares_World_current_close + 
                         iShares_SNP500_total_shares*iShares_SNP500_current_close + 
                         Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close + Cash_total)
        newrow = pd.DataFrame({'Date': [date], 'iShares_World_Value': [iShares_World_Value], 'iShares_SNP500_Value': [iShares_SNP500_Value],
                                "Vanguard_Global_Bond_Value":[Vanguard_Global_Bond_Value],"Cash_total":[Cash_total], 
                               "Total_Value": [Total_Value]}).set_index("Date")        
        df = pd.concat([df, newrow])
   
    results = df
    return results



In [9]:
#Repeat the same but rebalancing yearly, all we add is an if statement which once a year will in essence sell all the shares
#then reinvest at the original inputted proportions
def rebalancing_yearly(iShares_World_prop,iShares_SNP500_prop,Vanguard_Global_Bond_prop,Cash_prop):

    total_shares = 0
    total_invested = 0
    
    iShares_World_total_shares = 0
    iShares_SNP500_total_shares = 0
    Vanguard_Global_Bond_total_shares = 0
    Cash_total = 0
    Total_Value = 0
    
    df = pd.DataFrame({'Date': [], 'iShares_World_Value': [], 'iShares_SNP500_Value': [],
                       "Vanguard_Global_Bond_Value":[],"Cash_total":[], "Total_Value":[]}).set_index("Date")
    
    
    for date in date_range:
        if date in year_starts:
            #iShares World
            iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
            iShares_World_total_shares =  (Total_Value+100)*iShares_World_prop/iShares_World_current_close
            iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
            
            #iShares SNP500
            iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
            iShares_SNP500_total_shares =  (Total_Value + 100)*iShares_SNP500_prop/iShares_SNP500_current_close
            iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
            
            #Vanguard
            Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
            Vanguard_Global_Bond_total_shares = (Total_Value + 100)*Vanguard_Global_Bond_prop/Vanguard_Global_Bond_current_close
            Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
            
            #Cash
            BoE_current_rate = merged.loc[date,"BoE_Rate"]
            BoE_current_rate_monthly = BoE_current_rate/12
            Cash_total = (100+Total_Value)*Cash_prop

        else:
            #iShares World
            iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
            iShares_World_total_shares =  iShares_World_total_shares + (100*iShares_World_prop/iShares_World_current_close)
            iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
            
            #iShares SNP500
            iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
            iShares_SNP500_total_shares =  iShares_SNP500_total_shares + (100*iShares_SNP500_prop/iShares_SNP500_current_close)
            iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
            
            #Vanguard
            Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
            Vanguard_Global_Bond_total_shares =  Vanguard_Global_Bond_total_shares + (100*Vanguard_Global_Bond_prop/Vanguard_Global_Bond_current_close)
            Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
            
            #Cash
            BoE_current_rate = merged.loc[date,"BoE_Rate"]
            BoE_current_rate_monthly = BoE_current_rate/12
            Cash_total = (100*Cash_prop) + (BoE_current_rate_monthly*Cash_total/100) + Cash_total
        
        Total_Value=(iShares_World_total_shares*iShares_World_current_close + 
                         iShares_SNP500_total_shares*iShares_SNP500_current_close + 
                         Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close + Cash_total)
        newrow = pd.DataFrame({'Date': [date], 'iShares_World_Value': [iShares_World_Value], 'iShares_SNP500_Value': [iShares_SNP500_Value],
            "Vanguard_Global_Bond_Value":[Vanguard_Global_Bond_Value],"Cash_total":[Cash_total], "Total_Value": [Total_Value]}).set_index("Date")        
        df = pd.concat([df, newrow])
   
    results = df
    return results



### Running our functions and returning results
Next we create 4 options for porfolios and run these inside of our functions


In [11]:
#Creating a dataframe of 4 porfolio weightings
Proportions_df = pd.DataFrame({"iShares_World_prop": [0.45, 0.6, 0.7,0.8], 
                               "Vanguard_Global_Bond_prop": [0.45, 0.3, 0.2, 0.1], "Cash_prop":[0.1,0.1,0.1, 0.1]})
Proportions_df

,iShares_World_prop,Vanguard_Global_Bond_prop,Cash_prop
0,0.45,0.45,0.1
1,0.60,0.30,0.1
2,0.70,0.20,0.1
3,0.80,0.10,0.1


In [12]:
#We want to run our invest monthly fuction with each of these 4 portfolios

#For ease we create the first row of results first with the weights at row indexed 0
first_row_data = invest_monthly(Proportions_df.iloc[0][0], 0, Proportions_df.iloc[0][1], Proportions_df.iloc[0][2])
results_df = pd.DataFrame(first_row_data.tail(1))

#Then we cycle through the remaining rows
for i in range(1, len(Proportions_df)):
    data = invest_monthly(Proportions_df.iloc[i][0], 0, Proportions_df.iloc[i][1], Proportions_df.iloc[i][2])
    newrow = data.tail(1)

    #Adding each row to our results in results_df
    results_df = pd.concat([results_df, newrow])

#New we drop the unwanted column, reset the index and merge with the proportions dataframe to create our results with the associated weightings
results_df = results_df.drop(columns =["iShares_SNP500_Value"])
results_df = results_df.reset_index(drop=True)
invest_monthly_results = pd.merge(results_df, Proportions_df, left_index=True, right_index=True)

In [13]:
#Same again but repeated with our rebalancing formula
first_row_data = rebalancing_yearly(Proportions_df.iloc[0][0], 0, Proportions_df.iloc[0][1], Proportions_df.iloc[0][2])
results_df = pd.DataFrame(first_row_data.tail(1))


for i in range(1, len(Proportions_df)):
    data = rebalancing_yearly(Proportions_df.iloc[i][0], 0, Proportions_df.iloc[i][1], Proportions_df.iloc[i][2])
    newrow = data.tail(1)


    results_df = pd.concat([results_df, newrow])

results_df = results_df.drop(columns =["iShares_SNP500_Value"])
results_df = results_df.reset_index(drop=True)
yearly_rebalance_results = pd.merge(results_df, Proportions_df, left_index=True, right_index=True)

### Creating a normalised dataset

For our Power BI analysis it is helpful to have a normalised set of data, based on our original imported dataset of our indicied and how they performed

In [15]:
#Creating a normalied dataset from our original data to be used in graphs later

#Restricting to our desired dates and columns
merged = merged.loc[date_range]
merged = merged.drop(columns = ["BoE_Rate"])

#Next we create our normalised data
normalised_dataset = pd.DataFrame([])

for column in ["iShares_World_Close",	"iShares_SNP500_Close",	"Vanguard_Global_Bond_Close"]:
    #Cycling through each column, we divide by the first entry in each column so all the assets start at 1
    normalised_dataset[column] = merged[column] / merged[column][0]


### Exporting our results

In [14]:
#Exporting our results to csv files
yearly_rebalance_results.to_csv("Exported_Data//Yearly Rebalanced Results.csv")
invest_monthly_results.to_csv("Exported_Data//Invest Monthly Results.csv")

In [16]:
#Exporting our normalised data to a csv file
normalised_dataset.to_csv("Exported_Data//Normalised Diverse Portfolio.csv")